<a href="https://colab.research.google.com/github/lincolnschick/ML4MC/blob/main/docs/reports/requirement-48/final_combat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parameters

In [ ]:
#Variables to change based upon specifics of test
TEST_EPISODES = 1 # Number of tests to run for each model
TEST_STEPS = 10000 # Total timesteps to run for each model
USING_CUSTOM_ENV = True #Are we using a custom enviroment
DIRECTORY_PATH = "/content/drive/MyDrive/packages/minerl_saved_models" #Directory we have the models saved in
SAVE_LOCATION = "/content/drive/MyDrive/packages/minerl_test_outputs" #Directory we are saving videos to
FORCE_STOP = False #Force stops after one test (for code testing purposes)

#Installations

In [ ]:
import sys
from google.colab import drive
# Allow colab to access google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
LOCAL_MINERL = True

if LOCAL_MINERL:
  !chmod 555 -R "/content/drive/MyDrive/packages/minerl"
  sys.path.append("/content/drive/MyDrive/packages/minerl")
  !chmod 555 -R "/content/drive/MyDrive/packages/MixinGradle-dcfaf61"
  sys.path.append("/content/drive/MyDrive/packages/MixinGradle-dcfaf61")


In [ ]:
%%capture
!sudo add-apt-repository -y ppa:openjdk-r/ppa
!sudo apt-get purge openjdk-*
!sudo apt-get install openjdk-8-jdk
!sudo apt-get install xvfb
!sudo apt-get install xserver-xephyr
!sudo apt install tigervnc-standalone-server
!sudo apt-get install -y python3-opengl
!sudo apt-get install ffmpeg
!pip3 install gym==0.13.1
if LOCAL_MINERL:
  !pip3 install -e /content/drive/MyDrive/packages/minerl
else:
  !pip3 install minerl==0.4.4 --verbose
!pip3 install pyvirtualdisplay
!pip3 install -U colabgymrender
!sudo apt-get install xvfb
!pip3 install opencv-python
!pip3 install imageio==2.4.1

# Custom Environment Setup

In [ ]:
from minerl.herobraine.env_specs.simple_embodiment import SimpleEmbodimentEnvSpec
from minerl.herobraine.hero.handler import Handler
from typing import List
import random

import minerl.herobraine.hero.handlers as handlers
from minerl.herobraine.hero.mc import ALL_ITEMS


"""
The intent of this env_spec is to create a survival environment for our agent to be evaluated in.
This environment allows us to tailor the observation and action spaces to our agent's and UI's needs.
"""

NONE = 'none'
OTHER = 'other'

MS_PER_STEP = 50

ML4MC_SURVIVAL_LENGTH = 1 * 60 * 60 * 20  # 1 hour * 60 minutes * 60 seconds * 20 ticks/steps per second

class ML4MCSurvival(SimpleEmbodimentEnvSpec):
    # ML4MCSurvival constructor
    def __init__(self, *args, **kwargs):
        if 'name' not in kwargs:
            kwargs['name'] = 'ML4MCSurvival-v0' # Add environment name if not added

        super().__init__(*args, max_episode_steps=ML4MC_SURVIVAL_LENGTH, **kwargs)
        self.mission_xml = self.create_mission()

    # Allows scripts to observe inventory, equipped item, and current location related stats
    def create_observables(self) -> List[Handler]:
        return super().create_observables() + [
            handlers.FlatInventoryObservation(ALL_ITEMS),
            handlers.EquippedItemObservation(items=[
                'air', 'wooden_axe', 'wooden_pickaxe', 'stone_axe', 'stone_pickaxe', 'iron_axe', 'iron_pickaxe', 'diamond_sword', 'diamond_helmet', 'diamond_chestplate', 'diamond_leggings',
                'diamond_boots', NONE,
                OTHER
            ], _default='air', _other=OTHER),
            handlers.ObservationFromCurrentLocation(),
            handlers.ObservationFromLifeStats(),
        ]

    # Allows scripts to place blocks, equip items, craft items, and smelt items
    def create_actionables(self):
        return super().create_actionables() + [
            handlers.PlaceBlock([NONE, 'dirt', 'stone', 'cobblestone', 'crafting_table', 'furnace', 'torch'],
                                _other=NONE, _default=NONE),
            handlers.EquipAction([NONE, 'air', 'wooden_axe', 'stone_axe', 'iron_axe', 'stone_sword', 'iron_sword', 'wooden_sword', 'diamond_sword',
                                  'diamond_helmet', 'diamond_chestplate', 'diamond_leggings', 'diamond_boots',], _other=NONE, _default=NONE),
            handlers.CraftAction([NONE, 'torch', 'stick', 'planks', 'crafting_table'], _other=NONE, _default=NONE),
            handlers.CraftNearbyAction(
                [NONE, 'wooden_axe', 'wooden_pickaxe', 'stone_axe', 'stone_pickaxe', 'iron_axe', 'iron_pickaxe',
                 'furnace'], _other=NONE, _default=NONE),
            handlers.SmeltItemNearby([NONE, 'iron_ingot', 'coal'], _other=NONE, _default=NONE),
        ]

    # Rewards for collecting iron (and cobblestone)
    def create_rewardables(self) -> List[Handler]:
        return [
            # XP Reward
            handlers.RewardForXPGain(
                reward_per_xp=1000.0,
                reward_type="FIXED"),
            # Damage Negative Reward
            handlers.RewardForTakingDMG(
                reward_per_dmg=7.0,
                reward_type="FIXED"),
            # Survival Reward
            handlers.RewardForSurviving(
                reward_per_tick=5.0),
            # Not Walking Reward
            handlers.RewardForNotWalking(
                reward_per_tick=5.0),
            handlers.RewardForTouchingBlockType([
                {'type':'obsidian', 'behaviour':'onceOnly', 'reward':1},
            ]),
            handlers.RewardForCollectingItems([
                dict(type="diamond_sword", amount=1, reward=2.0),
                dict(type="rotten_flesh", amount=1, reward=50.0),
            ])

        ]

    def create_mission(self):
      return f'''
      <Mission>
          <!-- Other mission parameters -->

          <RewardForDamagingEntity>
              <Mob type="Zombie" reward="40"/>
          </RewardForDamagingEntity>

          <AgentHandlers>
              <!-- Your agent handlers -->
          </AgentHandlers>
      </Mission>
      '''



    # Start the agent with nothing by default, can be modified for testing
    def create_agent_start(self) -> List[Handler]:
        return [
            handlers.SimpleInventoryAgentStart([
                dict(type="diamond_sword", quantity=5),
                dict(type="diamond_helmet", quantity=1),
                dict(type="diamond_chestplate", quantity=1),
                dict(type="diamond_leggings", quantity=1),
                dict(type="diamond_boots", quantity=1)
            ]),
            handlers.AgentStartPlacement(0, 11, 0, 0, 0)
        ]

    # No agent handlers needed as we are not using any rewards
    def create_agent_handlers(self) -> List[Handler]:
        return [
            handlers.AgentQuitFromPossessingItem([
                dict(type="diamond_ore", amount=32)]
            )
        ]

    # Use the default world generator
    def create_server_world_generators(self) -> List[Handler]:
        # return [handlers.BiomeGenerator("extreme_hills")]
        return [
            handlers.FlatWorldGenerator(generatorString="1;7,2x3,2;1"),
            handlers.DrawingDecorator("""
              <DrawCuboid x1="-5" y1="7" z1="-5" x2="5" y2="7" z2="5" type="obsidian"/>
              <DrawCuboid x1="-6" y1="8" z1="-6" x2="6" y2="9" z2="6" type="fence"/>
              <DrawCuboid x1="-5" y1="8" z1="-5" x2="5" y2="9" z2="5" type="air"/>
              <DrawBlock x="0" y="8" z="3" type="fence_gate"/>
              <DrawBlock x="0" y="9" z="3" type="fence_gate"/>
              <DrawCuboid x1="-4" y1="8" z1="1" x2="6" y2="8" z2="1" type="fence"/>
              <DrawEntity x="0" y="10" z="2" type="Zombie"/>
          """)

        ]

    def create_server_quit_producers(self) -> List[Handler]:
        # Set a timeout to end the episode to prevent it from running forever
        return [
            handlers.ServerQuitFromTimeUp(time_limit_ms=self.max_episode_steps * MS_PER_STEP),
            handlers.ServerQuitWhenAnyAgentFinishes()
        ]

    # This method can be used to change other things about the world such as drawing shapes or spawning a village
    # Not needed for ML4MCSurvival
    def create_server_decorators(self) -> List[Handler]:
        return [
            # Attempt at creating an enclosed space
            # handlers.DrawingDecorator("""
            #   <DrawCuboid type="bedrock" x1="-15" x2="16" y1="1" y2="50" z1="-15" z2="16" />
            #   <DrawCuboid type="air" x1="-12" x2="13" y1="4" y2="39" z1="-12" z2="13" />
            #   <DrawCuboid type="glowstone" x1="-14" x2="15" y1="7" y2="30" z1="-14" z2="15" />
            # """)

            handlers.DrawingDecorator("""
              <DrawCuboid type="bedrock" x1="-15" x2="14" y1="-10" y2="50" z1="16" z2="16" />
              <DrawCuboid type="bedrock" x1="-15" x2="16" y1="-10" y2="50" z1="-15" z2="-15" />
              <DrawCuboid type="bedrock" x1="-15" x2="-15" y1="-10" y2="50" z1="-15" z2="16" />
              <DrawCuboid type="bedrock" x1="16" x2="16" y1="-10" y2="50" z1="-15" z2="16" />

            """)
        ]

    # This method sets the conditions for the world the agent will spawn into
    # We will allow spawning and the passage of time to replicate a realistic Minecraft environment
    def create_server_initial_conditions(self) -> List[Handler]:
        return [
            handlers.TimeInitialCondition(
                start_time=15000, #18000 is night #6000 is day
                allow_passage_of_time=True,
            ),
            handlers.SpawningInitialCondition(
                allow_spawning=True
            )
        ]

    def is_from_folder(self, folder: str) -> bool:
        return folder == 'ml4mc_survival'

    # Don't need docstring as we're not publishing this environment to MineRL's website
    def get_docstring(self):
        return ""

    def determine_success_from_rewards(self, rewards: list) -> bool:
        # All survival experiemnts are a success =)
        return sum(rewards) >= self.reward_threshold

#Setup

In [ ]:

import os
import numpy as np
import torch as th
from torch import nn
import gym
import minerl
import pandas as pd
from tqdm.notebook import tqdm
from colabgymrender.recorder import Recorder
from pyvirtualdisplay import Display
import logging
logging.disable(logging.ERROR)
from datetime import datetime

In [ ]:
#Start the Display for saving videos on Colab
from pyvirtualdisplay import Display
from os import path
display = Display(visible=False, size=(400, 300))
display.start();

In [ ]:
class NatureCNN(nn.Module):
    """
    CNN from DQN nature paper:
        Mnih, Volodymyr, et al.
        "Human-level control through deep reinforcement learning."
        Nature 518.7540 (2015): 529-533.

    :param input_shape: A three-item tuple telling image dimensions in (C, H, W)
    :param output_dim: Dimensionality of the output vector
    """

    def __init__(self, input_shape, output_dim):
        super().__init__()
        n_input_channels = input_shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(th.zeros(1, *input_shape)).shape[1]

        self.linear = nn.Sequential(
            nn.Linear(n_flatten, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))

In [ ]:
class ActionShaping(gym.ActionWrapper):
  def __init__(self, env, camera_angle=10, always_attack=False):
    super().__init__(env)

    self.camera_angle = camera_angle
    self.always_attack = always_attack
    self._actions = [
      [('attack', 1)],
      [('forward', 1)],
      # [('back', 1)],
      # [('left', 1)],
      # [('right', 1)],
      # [('jump', 1)],
      # [('forward', 1), ('attack', 1)],
      # [('craft', 'planks')],
      [('forward', 1), ('jump', 1)],
      [('camera', [-self.camera_angle, 0])],
      [('camera', [self.camera_angle, 0])],
      [('camera', [0, self.camera_angle])],
      [('camera', [0, -self.camera_angle])],
    ]

    self.actions = []
    for actions in self._actions:
      act = self.env.action_space.noop()
      for a, v in actions:
        act[a] = v
      if self.always_attack:
        act['attack'] = 1
      self.actions.append(act)

    self.action_space = gym.spaces.Discrete(len(self.actions))

  def action(self, action):
    return self.actions[action]

In [ ]:
def dataset_action_batch_to_actions(dataset_actions, camera_margin=5):
  # There are dummy dimensions of shape one
  camera_actions = dataset_actions["camera"].squeeze()
  attack_actions = dataset_actions["attack"].squeeze()
  forward_actions = dataset_actions["forward"].squeeze()
  jump_actions = dataset_actions["jump"].squeeze()
  batch_size = len(camera_actions)
  actions = np.zeros((batch_size,), dtype=np.int)

  for i in range(len(camera_actions)):
    # Moving camera is most important (horizontal first)
    if camera_actions[i][0] < -camera_margin:
      actions[i] = 3
    elif camera_actions[i][0] > camera_margin:
      actions[i] = 4
    elif camera_actions[i][1] > camera_margin:
      actions[i] = 5
    elif camera_actions[i][1] < -camera_margin:
      actions[i] = 6
    elif forward_actions[i] == 1:
      if jump_actions[i] == 1:
        actions[i] = 2
      else:
        actions[i] = 1
    elif attack_actions[i] == 1:
      actions[i] = 0
    else:
      # No reasonable mapping (would be no-op)
      actions[i] = -1
  return actions

In [ ]:
def str_to_act(env, actions):
  act = env.action_space.noop()
  for action in actions.split():
    if ":" in action:
      k, v = action.split(':')
      if k == 'camera':
        act[k] = eval(v)
      else:
        act[k] = v
    else:
      act[action] = 1
  return act

In [ ]:
abs_CUSTOM = ML4MCSurvival()
abs_CUSTOM.register()

In [ ]:
!pip3 install stable-baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

Training

In [ ]:
from stable_baselines3.common import results_plotter
from stable_baselines3.common import monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
from stable_baselines3 import PPO

env = gym.make('ML4MCSurvival-v0')


In [ ]:
from colabgymrender.recorder import Recorder
env = Recorder(env, "/content/drive/MyDrive/ml4mc_outputs/sunday", fps=60)

In [ ]:
!pip install shimmy

In [ ]:
#A wrapper for getting the POV of the avatar from the environment, which is needed for stable_baselines
class ExtractPOV(gym.ObservationWrapper):
  def __init__(self, env):
    super().__init__(env)
    self.observation_space = self.env.observation_space['pov']

  def observation(self, observation):
    return observation['pov']

In [ ]:
# y = int(obs['location_stats']['ypos'])

In [ ]:
"""
#callback addition
log_dir = "tmp/"
os.makedirs(log_dir, exist_ok=True)
monitored_env = monitor.Monitor(env1, log_dir)
callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)
"""
obs_wrapped_combat = ExtractPOV(env) #Extracting the POV of the avatar from the environment which is needed for stable_baselines
obs_action_wrapped_combat = ActionShaping(obs_wrapped_combat) #Performing action shaping on the actions of the environment to convert them from dictionaries into an array.
obs = obs_action_wrapped_combat.reset() #reseting the provided environnment

model = PPO(policy="CnnPolicy", env=obs_action_wrapped_combat, verbose=1) #Setting the model to be a PPO model with a CnnPolicy. This was just the model used by tutorials, we'll experiment with the best model later
model.learn(total_timesteps=50000) #Training the model, allowing it to walk through 50000 timesteps of the environment (about 1.5 minutes)
# model.learn(total_timesteps=100000000)
env.release() #releasing the recorded environment to actually make a video on Colab.


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 674      |
|    ep_rew_mean     | 1        |
| time/              |          |
|    fps             | 8        |
|    iterations      | 1        |
|    time_elapsed    | 241      |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 674         |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 9           |
|    iterations           | 2           |
|    time_elapsed         | 412         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015719822 |
|    clip_fraction        | 0.193       |
|    clip_range      

In [ ]:
"""
results_plotter.plot_results([log_dir], 5000, results_plotter.X_TIMESTEPS, "MineRL RL Training")
plt.show()
"""
model.save(DIRECTORY_PATH + "/" + 'combat.pth' )

In [ ]:
model.save(DIRECTORY_PATH + "/" + 'combat2.pth' )